<a href="https://colab.research.google.com/github/physicaone/loss_IG/blob/master/%5B210802%5DExact_quantities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base='drive/MyDrive'
except:
    base='Google Drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import torch.utils.data
import torch.nn as nn
from itertools import combinations
from itertools import permutations
from datetime import datetime
from tqdm import tqdm, tnrange
import warnings
warnings.filterwarnings("ignore")
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
# np.seterr(divide='ignore', invalid='ignore')
import itertools
import re
from time import sleep
from scipy.stats import entropy


#Define RBM class

In [3]:
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid, k, use_cuda):
        """Create a RBM."""
        super(RBM, self).__init__()
        
        if use_cuda==True:
            self.v = nn.Parameter(torch.ones(1, n_vis).cuda())
            self.h = nn.Parameter(torch.zeros(1, n_hid).cuda())
            self.W = nn.Parameter(torch.randn(n_hid, n_vis).cuda())
            self.k = k
        else:
            self.v = nn.Parameter(torch.ones(1, n_vis))
            self.h = nn.Parameter(torch.zeros(1, n_hid))
            self.W = nn.Parameter(torch.randn(n_hid, n_vis))
            self.k = k     

    def visible_to_hidden(self, v, beta):
        return torch.sigmoid(F.linear(v, self.W, self.h)*beta)

    def hidden_to_visible(self, h, beta):
        return torch.sigmoid(F.linear(h, self.W.t(), self.v)*beta)

    def free_energy(self, v):
        v_term = torch.matmul(v, self.v.t())
        w_x_h = F.linear(v, self.W, self.h)
        h_term = torch.sum(F.softplus(w_x_h), dim=1)
        return torch.mean(-h_term - v_term)
    
    def energy2(self, v, h):
        return -torch.matmul(v, self.v.t())-torch.matmul(torch.matmul(v, self.W.t()),h.t())-torch.matmul(h, self.h.t())

    def forward(self, v):
        h = self.visible_to_hidden(v)
        h = h.bernoulli()
        for _ in range(self.k):
            v_gibb = self.hidden_to_visible(h)
            v_gibb = v_gibb.bernoulli()
            h = self.visible_to_hidden(v_gibb)
            h = h.bernoulli()
        return v, v_gibb

# Check CUDA availability

In [4]:
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

if CUDA:
    device='cuda'
else:
    device='cpu'
torch.cuda.is_available(); CUDA

True

# Define fundamental functions

In [5]:
# 아래 두 함수는 PT와는 아무 상관 없습니다. 10진법을 2진법으로 바꾸고 리스트 형태로 변형하는 함수입니다. 예)15->[1,1,1,1]
# 그리고 그 다음은 역함수입니다.
def decimal_to_binary(integer, n_hid):
    string=bin(int(integer))[2:]
    list0=[float(d) for d in string]
    while len(list0)<n_hid:
        list0=[0.]+list0
    return torch.tensor([list0])

def binary_to_decimal(list0):
    value=0
    list0=list0.tolist()
    for i in range(len(list0)):
        value+=list0[-i-1]*2**(i)
    return int(value)

def get_hist(list00, color='red'):
    bins=range(int(min(list00)-30), int(max(list00)+30), 1)
    y1,x1,_ = plt.hist(list00, bins = bins, histtype='step', color=color)
    x1 = 0.5*(x1[1:]+x1[:-1])
    return x1, y1

def flatten_list(list0):
    flattened = [val for sublist in list0 for val in sublist]
    return flattened

def Energy(model0_dict, v_list, h_list):
    a=model0_dict['v'].detach()
    b=model0_dict['h'].detach()
    W=model0_dict['W'].detach()
    values=[]
    for i in range(len(v_list)):
        e=-np.matmul(v_list[i], a.t())-np.matmul(np.matmul(v_list[i], W.t()), h_list[i].t())-np.matmul(h_list[i], b.t())
        values.append(e.detach())
    return float(np.mean(values))
    
def Energy_GPU(model0_dict, v_list0, h_list0):
    n_split=4
    a=model0_dict['v'].detach().cuda().view(n_vis)
    b=model0_dict['h'].detach().cuda().view(n_hid)
    W=model0_dict['W'].detach().cuda().view(n_hid, n_vis)
    values=[]
    len(h_list0)
    for i in range(n_split):
        v_list=torch.stack(list(v_list0[i*int(len(v_list0)/n_split):(i+1)*int(len(v_list0)/n_split)]))
        h_list=torch.stack(list(h_list0[i*int(len(h_list0)/n_split):(i+1)*int(len(h_list0)/n_split)]))
        v_list=v_list.detach().cuda().view(len(v_list), n_vis)
        h_list=h_list.detach().cuda().view(len(h_list), n_hid)
        e=-torch.matmul(v_list.float(), a)-torch.diagonal(torch.matmul(torch.matmul(v_list.float(), W.t()), h_list.float().t()))-torch.matmul(h_list.float(), b)
        values.append(np.mean(e.cpu().detach().numpy()))
    return np.mean(values)

# Energy_GPU returns an array of values
def Energy_GPU2(model0_dict, v_list0, h_list0):
    n_split=4
    a=model0_dict['v'].detach().cuda().view(n_vis)
    b=model0_dict['h'].detach().cuda().view(n_hid)
    W=model0_dict['W'].detach().cuda().view(n_hid, n_vis)
    values=[]
    len(h_list0)
    for i in range(n_split):
        v_list=torch.stack(list(v_list0[i*int(len(v_list0)/n_split):(i+1)*int(len(v_list0)/n_split)]))
        h_list=torch.stack(list(h_list0[i*int(len(h_list0)/n_split):(i+1)*int(len(h_list0)/n_split)]))
        v_list=v_list.detach().cuda().view(len(v_list), n_vis)
        h_list=h_list.detach().cuda().view(len(h_list), n_hid)
        e=-torch.matmul(v_list.float(), a)-torch.diagonal(torch.matmul(torch.matmul(v_list.float(), W.t()), h_list.float().t()))-torch.matmul(h_list.float(), b)
        values.append(e.cpu().detach().numpy())

    return np.array(flatten_list(values))


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def Exact_Q_Z(model0_dict, n_vis, n_hid, n_split, v_list0, h_list0):
    a=model0_dict['v'].detach().cuda().view(n_vis)
    b=model0_dict['h'].detach().cuda().view(n_hid)
    W=model0_dict['W'].detach().cuda().view(n_hid, n_vis)
    values=[]
    len(h_list0)
    for i in range(n_split):
        v_list=torch.stack(list(v_list0[i*int(len(v_list0)/n_split):(i+1)*int(len(v_list0)/n_split)]))
        h_list=torch.stack(list(h_list0[i*int(len(h_list0)/n_split):(i+1)*int(len(h_list0)/n_split)]))
        v_list=v_list.detach().cuda().view(len(v_list), n_vis)
        h_list=h_list.detach().cuda().view(len(h_list), n_hid)
        e=-torch.matmul(v_list.float(), a)-torch.diagonal(torch.matmul(torch.matmul(v_list.float(), W.t()), h_list.float().t()))-torch.matmul(h_list.float(), b)
        values=values+list(torch.exp(-e).detach().cpu().numpy())

    # return float(torch.sum(torch.tensor(values)).detach().numpy())
    return np.array(values)/sum(values), sum(values)

def Energy_from_Q(Q_Z_ideal, Q_Z_real):
    ideal_E_list=-np.log(np.array(Q_Z_ideal[0]*Q_Z_ideal[1]))
    real_E_list=-np.log(np.array(Q_Z_real[0]*Q_Z_ideal[1]))
    dE=[]
    for i in range(len(ideal_E_list)):
        dE.append(real_E_list[i]*Q_Z_ideal[0][i]-ideal_E_list[i]*Q_Z_ideal[0][i])
    return sum(dE)


In [6]:
n_hid_list=[16,24,32]
T_list=[1.47,1.78,2.3,5.2,16]
n_vis=9

# Exact distribution and partition function

In [ ]:
for n_hid in [1,2,4,8,12]:
    v_list=[]; h_list=[]
    for s in range(2**(n_vis+n_hid)):
        full=decimal_to_binary(s, n_hid+n_vis)[0]
        v=full[:n_vis]; h=full[-n_hid:]
        v_list.append(v); h_list.append(h)
    v_list0=torch.stack(v_list)
    h_list0=torch.stack(h_list)
    if n_hid in [1,2,4,8]:
        n_split=4
    else:
        n_split=2000
    for T in T_list:
        dict0={}; dict1={}; dict2={}
        models=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_mn_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T)) 
        model_dicts_CM_m=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM_m_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T))
        model_dicts_CM=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T))
        dict2=Exact_Q_Z(model_dicts_CM, n_vis, n_hid, n_split, v_list0, h_list0)
        for m in tnrange(10):
            dict0[str(m)]={}
            dict1[str(m)]=Exact_Q_Z(model_dicts_CM_m[str(m)], n_vis, n_hid, n_split, v_list0, h_list0)
            for n in range(10):
                dict0[str(m)][str(n)]=Exact_Q_Z(models[str(m)][str(n)], n_vis, n_hid, n_split, v_list0, h_list0)
        
        with open('{base}/loss_IG/3*3/data/3*3_n_hid={n_hid}_T={T}_Q_Z_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump({'mn':dict0, 'CM_m':dict1, 'CM':dict2}, f)

In [ ]:
# exact FE using rearranged exact Z
for n_hid in [1,2,4,8,12]:
    for T in T_list:
        dict0={}; dict1={}; dict2={}
        models=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_mn_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T)) 
        model_dicts_CM_m=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM_m_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T))
        model_dicts_CM=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T))
        Q_Z=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_n_hid={n_hid}_T={T}_Q_Z_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T))
        dict2=-np.log(Q_Z['CM'][1])
        for m in tnrange(10):
            dict0[str(m)]={}
            dict1[str(m)]=-np.log(Q_Z['CM_m'][str(m)][1])
            for n in range(10):
                dict0[str(m)][str(n)]=-np.log(Q_Z['mn'][str(m)][str(n)][1])
        
        with open('{base}/loss_IG/3*3/data/3*3_n_hid={n_hid}_T={T}_FE_exact_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump({'mn':dict0, 'CM_m':dict1, 'CM':dict2}, f)

In [ ]:
# Energy differnece term using rearranged exact Q
for n_hid in [1,2,4,8,12]:
    for T in T_list:
        dict_CM={}; dict_CM_m={}
        Q_Z=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_n_hid={n_hid}_T={T}_Q_Z_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T))
        for m in tnrange(10):
            dict_CM[str(m)]={}
            dict_CM_m[str(m)]={ }
            for n in range(10):
                dict_CM[str(m)][str(n)]=Energy_from_Q(Q_Z['CM'], Q_Z['mn'][str(m)][str(n)])
                dict_CM_m[str(m)][str(n)]=Energy_from_Q(Q_Z['CM_m'][str(m)], Q_Z['mn'][str(m)][str(n)])
        
        with open('{base}/loss_IG/3*3/data/3*3_n_hid={n_hid}_T={T}_dE_exact_rearranged.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump({'CM_m':dict_CM_m, 'CM':dict_CM}, f)

# Exact FE for larger n_hid

In [ ]:
# Energy differnece term using rearranged exact Q
n_split=2**13

for n_hid in [16]:
    for T in [1.47,1.78,2.3,5.2,16]:
        model_dicts_mn=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_mn.pkl'.format(base=base, n_hid=n_hid, T=T)) 
        model_dicts_CM=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM.pkl'.format(base=base, n_hid=n_hid, T=T))

        dE_CM2mn={}
        dE_mn2CM={}
        for m in range(10):
            dE_CM2mn[str(m)]={}
            dE_mn2CM[str(m)]={}
            for n in range(10):
                dE_CM2mn[str(m)][str(n)]=0
                dE_mn2CM[str(m)][str(n)]=0
        
        FE_mn={}
        FE_CM=0
        for m in range(10):
            FE_mn[str(m)]={}
            for n in range(10):
                FE_mn[str(m)][str(n)]=0

        for j in tnrange(n_split):
            v_list=[]; h_list=[]
            for s in range(j*int(2**(n_vis+n_hid)/n_split), (j+1)*int(2**(n_vis+n_hid)/n_split)):
                full=decimal_to_binary(s, n_hid+n_vis)[0]
                v=full[:n_vis]; h=full[-n_hid:]
                v_list.append(v); h_list.append(h)
            v_list0=torch.stack(v_list)
            h_list0=torch.stack(h_list)

            E_CM=Energy_GPU2(model_dicts_CM, v_list0, h_list0)
            FE_CM+=sum(np.exp(-E_CM))
            for m in range(10):
                for n in range(10):
                    E_mn=Energy_GPU2(model_dicts_mn[str(m)][str(n)], v_list0, h_list0)
                    dE_mn2CM[str(m)][str(n)]+=sum((E_CM-E_mn)*np.exp(-E_mn))
                    dE_CM2mn[str(m)][str(n)]+=sum((E_mn-E_CM)*np.exp(-E_CM))
                    FE_mn[str(m)][str(n)]+=sum(np.exp(-E_mn))
        
        for m in range(10):
            for n in range(10):
                dE_CM2mn[str(m)][str(n)]=dE_CM2mn[str(m)][str(n)]/np.exp(-FE_mn[str(m)][str(n)])
                dE_mn2CM[str(m)][str(n)]=dE_mn2CM[str(m)][str(n)]/np.exp(-FE_CM)

        with open('{base}/loss_IG/3*3/data/3*3_n_hid={n_hid}_T={T}_FE_exact.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump({'mn':FE_mn, 'CM':FE_CM}, f)
        with open('{base}/loss_IG/3*3/data/3*3_n_hid={n_hid}_T={T}_dE_exact.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump({'mn2CM':dE_mn2CM, 'CM2mn':dE_CM2mn}, f)

# <E'-E> mn 서로 기준

In [8]:
# Energy differnece term using rearranged exact Q
n_split=2**13

for n_hid in [16]:
    for T in [1.47,1.78,2.3,5.2,16]:
        model_dicts_mn=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_mn.pkl'.format(base=base, n_hid=n_hid, T=T)) 
        FE_mn=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_T={T}_FE_exact.pkl'.format(base=base, T=T)['mn']
        dE_mn={}
        for m in range(10):
            dE_mn[str(m)]={}
            for n in range(10):
                dE_mn[str(m)][str(n)]=0
        

        for j in tnrange(n_split):
            v_list=[]; h_list=[]
            for s in range(j*int(2**(n_vis+n_hid)/n_split), (j+1)*int(2**(n_vis+n_hid)/n_split)):
                full=decimal_to_binary(s, n_hid+n_vis)[0]
                v=full[:n_vis]; h=full[-n_hid:]
                v_list.append(v); h_list.append(h)
            v_list0=torch.stack(v_list)
            h_list0=torch.stack(h_list)

            E_mn={}
            for m in range(10):
                E_mn[str(m)]={}
                for n in range(10):
                    E_mn[str(m)][str(n)]=Energy_GPU2(model_dicts_mn[str(m)][str(n)], v_list0, h_list0)

            dE_mn={}
            for i in tnrange(100):
                i=str(i).zfill(2)
                dE_mn[str(i)]={}
                for j in range(100):
                    j=str(j).zfill(2)
                    dE_mn[i][j]+=sum((E_mn[j[0]][j[1]]-E_mn[i[0]][i[1]])*np.exp(-E_mn[i[0]][i[1]]))

        for i in tnrange(100):
            i=str(i).zfill(2)
            for j in range(100):
                j=str(j).zfill(2)
                dE_mn[i][j]=dE_mn[i][j]/np.exp(-FE_mn[i[0]][i[1]])

        with open('{base}/loss_IG/3*3/data/3*3_n_hid={n_hid}_T={T}_dE10000_exact.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump(dE_mn, f)

1.78 12


# Entropy

In [ ]:
def Ising_energy(v_list):
    L = 3
    E_list=[]
    for n in range(len(v_list)):
        v=v_list[n]
        E = 0
        for i in range(L):
            for j in range(L):
                s = v[i,j]
                neigh = v[(i+1)%L, j] + v[i,(j+1)%L] + v[(i-1)%L,j] + v[i,(j-1)%L] 
                E += -neigh * s
        E_list.append(E/2)
    return np.array(E_list)


v_list=[]
for s in range(2**n_vis):
    v=decimal_to_binary(s, n_vis)[0]
    v_list.append(np.reshape(v,(3,3))*2-1)

for T in [1.47,1.78,2.3,5.2,16]:

    bf_list=np.exp(-Ising_energy(v_list)/T)
    Pv=bf_list/sum(bf_list)

    with open('{base}/loss_IG/3*3/data/3*3_T={T}_S_exact.pkl'.format(base=base, T=T), 'wb') as f:
        pkl.dump(entropy(Pv), f)
